In [ ]:
from utils import *
from tqdm.notebook import tqdm
from scipy import ndimage

res_1km = 1 / 12 / 10

# modis land cover color

In [ ]:
st1 = """Value	Color	Description
1	#05450a	Evergreen Needleleaf Forests: dominated by evergreen conifer trees (canopy >2m). Tree cover >60%.
2	#086a10	Evergreen Broadleaf Forests: dominated by evergreen broadleaf and palmate trees (canopy >2m). Tree cover >60%.
3	#54a708	Deciduous Needleleaf Forests: dominated by deciduous needleleaf (larch) trees (canopy >2m). Tree cover >60%.
4	#78d203	Deciduous Broadleaf Forests: dominated by deciduous broadleaf trees (canopy >2m). Tree cover >60%.
5	#009900	Mixed Forests: dominated by neither deciduous nor evergreen (40-60% of each) tree type (canopy >2m). Tree cover >60%.
6	#c6b044	Closed Shrublands: dominated by woody perennials (1-2m height) >60% cover.
7	#dcd159	Open Shrublands: dominated by woody perennials (1-2m height) 10-60% cover.
8	#dade48	Woody Savannas: tree cover 30-60% (canopy >2m).
9	#fbff13	Savannas: tree cover 10-30% (canopy >2m).
10	#b6ff05	Grasslands: dominated by herbaceous annuals (<2m).
11	#27ff87	Permanent Wetlands: permanently inundated lands with 30-60% water cover and >10% vegetated cover.
12	#c24f44	Croplands: at least 60% of area is cultivated cropland.
13	#a5a5a5	Urban and Built-up Lands: at least 30% impervious surface area including building materials, asphalt and vehicles.
14	#ff6d4c	Cropland/Natural Vegetation Mosaics: mosaics of small-scale cultivation 40-60% with natural tree, shrub, or herbaceous vegetation.
15	#69fff8	Permanent Snow and Ice: at least 60% of area is covered by snow and ice for at least 10 months of the year.
16	#f9ffa4	Barren: at least 60% of area is non-vegetated barren (sand, rock, soil) areas with less than 10% vegetation.
17	#1c0dff	Water Bodies: at least 60% of area is covered by permanent water bodies."""

class_table = pd.read_csv(StringIO(st1), sep="\t")
class_table["Value"] = class_table["Value"].astype(int)
class_table["luc"] = class_table["Description"].str.split(":", expand=True)[0]

code2color = class_table.set_index("Value")["Color"].to_dict()

In [ ]:
def get_gdf_conflict_1year(year, exclude=[]):
    shp_file = path_data / f"PSM/sample_point/conflict_{year}.shp"
    gdf_conflict_1year = gpd.read_file(shp_file)
    return gdf_conflict_1year
def get_miss_conflict_random_point(year, gdf_conflict_1year, non_conflict_multiple_index=2):
    shp_file = path_data / f"PSM/sample_point/non_conflict_{year}.shp"
    gdf_miss_conflict = gpd.read_file(shp_file)
    return gdf_miss_conflict

## circle mask

In [ ]:
# For subsequent sensitivity analysis, the sample radius has been extended to 20 km
radius, radius_b = 25, 24.43

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def get_best_clip_radius(radius, buffer_steps):
    gdf_point = gpd.GeoDataFrame({}, geometry=gpd.points_from_xy([0], [0]), crs="epsg:4326")

    min_diff = float('inf')
    best_step = None
    for step in buffer_steps:
        gdf_circle = gdf_point.copy()
        gdf_circle['geometry'] = gdf_point.buffer(radius + step)

        da_circle = xr.DataArray(np.ones((radius * 2 + 1, radius * 2 + 1)), coords={"y": np.arange(-radius, radius + 1), "x": np.arange(-radius, radius + 1)})\
            .rio.write_crs("epsg:4326")\
            .rio.clip(gdf_circle.geometry, all_touched=True, drop=False)\
            .fillna(0).astype(np.uint8)

        # Check the symmetry of the sample
        check_ar = np.array([
            ((da_circle.values == da_circle.values.T).sum() == (radius * 2 + 1) ** 2), 
            ((da_circle.values == da_circle.values[::-1]).sum() == (radius * 2 + 1) ** 2), 
            ((da_circle.values == da_circle.values.T[::-1]).sum() == (radius * 2 + 1) ** 2), 
        ])
        if (check_ar.sum() != 3):
            continue

        area_diff = abs(da_circle.sum().values - radius ** 2 * np.pi)

        if area_diff <= min_diff:
            min_diff = area_diff
            best_step = step
        else:
            break
    return radius + best_step, min_diff

In [ ]:
from tqdm.notebook import tqdm
buffer_steps = np.arange(-1, 1, 0.01)
radius_data = []
for radius in tqdm(np.arange(3, 26)):
    radius_clip, min_diff = get_best_clip_radius(radius, buffer_steps)
    radius_data.append([radius, radius_clip, min_diff])
df_radius = pd.DataFrame(radius_data, columns=["radius", "radius_clip", "diff"])

In [ ]:
def get_ar_circle(radius):
    radius_clip = {3: 2.54, 4:3.53, 5: 4.49, 6: 5.52, 7: 6.52, 8: 7.49, 9: 8.52, 10: 9.49, 11: 10.52, 12: 11.49, 13: 12.49, 14: 13.51, 15: 14.49, 16: 15.52, 17: 16.46, 18: 17.49, 19: 18.49, 20: 19.47, 21: 20.49, 22: 21.49, 23: 22.49, 24: 23.49, 25: 24.43}[radius]
    gdf_circle = gpd.GeoDataFrame({}, geometry=gpd.GeoDataFrame({}, geometry=gpd.points_from_xy([0], [0])).buffer(radius_clip), crs="epsg:4326")
    da_circle = xr.DataArray(np.ones((radius * 2 + 1, radius * 2 + 1)), coords={"y": np.arange(-radius, radius + 1), "x": np.arange(-radius, radius + 1)})\
        .rio.write_crs("epsg:4326")\
        .rio.clip(gdf_circle.geometry, all_touched=True, drop=False)\
        .fillna(0).astype(np.uint8)
    return da_circle, da_circle.values

# extract circle sample

In [ ]:
# MODIS MCD12Q1 
def load_luc(year):
    da_luc = xr.open_dataarray(path_data / f"luc/2001-2023-1km.tif")
    da_luc["band"] = [int(i) for i in ["0", *da_luc.attrs["long_name"][1:]]]
    da_luc = da_luc.sel(band=slice(year-1, year))
    da_luc.attrs = {}
    return da_luc

# MODIS MOD17A3HGF 
def load_npp(year):
    da_npp_1year = xr.open_dataarray(path_data / f"npp/{year}-1km.tif")\
        .sel(band=1).drop_vars(["band", "spatial_ref"])\
        .rio.write_crs("epsg:4326").rio.clip(gdf_world.geometry)\
        .fillna(0) * 0.1 # gC/m2/yr
    da_npp_lastyear = xr.open_dataarray(path_data / f"npp/{year-1}-1km.tif")\
        .sel(band=1).drop_vars(["band", "spatial_ref"])\
        .rio.write_crs("epsg:4326").rio.clip(gdf_world.geometry)\
        .fillna(0) * 0.1 # gC/m2/yr
    return da_npp_1year, da_npp_lastyear

In [ ]:
# diatance to road
da_road_dis = xr.open_dataarray(path_data / "PSM/dis_2_road.tif").sel(band=1).drop(["band", "spatial_ref"])
# distance to country boundary
da_boundary_dis = xr.open_dataarray(path_data / "PSM/dis_2_boundary.tif").sel(band=1).drop(["band", "spatial_ref"])

In [ ]:
def get_circle_locations(y, x, da_luc, radius):
    nearest_y = np.abs(da_luc.y - y).argmin().item()
    nearest_x = np.abs(da_luc.x - x).argmin().item()
    y_ = np.arange(nearest_y-radius, nearest_y+radius+1)
    x_ = np.arange(nearest_x-radius, nearest_x+radius+1)
    return y_, x_

def extract_circle(gdf_conflict_1year, da_luc, da_road_dis, da_boundary_dis, da_npp_1year, da_npp_lastyear, radius):
    da_luc_lastyear_dic = {}
    da_luc_1year_dic = {}
    da_road_distance_dic = {}
    da_boundary_dis_dic = {}
    da_npp_1year_dic = {}
    da_npp_lastyear_dic = {}

    for idx, x, y in tqdm(gdf_conflict_1year[["idx", "x", "y"]].values):

        y_, x_ = get_circle_locations(y, x, da_luc, radius)
        da_luc_lastyear_dic[idx] = da_luc.isel(band=0, x=x_, y=y_).drop("band")
        da_luc_1year_dic[idx] = da_luc.isel(band=1, x=x_, y=y_).drop("band")

        y_, x_ = get_circle_locations(y, x, da_road_dis, radius)
        da_road_distance_dic[idx] = da_road_dis.isel(x=x_, y=y_)
        da_boundary_dis_dic[idx] = da_boundary_dis.isel(x=x_, y=y_)

        y_, x_ = get_circle_locations(y, x, da_npp_1year, radius)
        da_npp_1year_dic[idx] = da_npp_1year.isel(x=x_, y=y_)
        da_npp_lastyear_dic[idx] = da_npp_lastyear.isel(x=x_, y=y_)
        

    return (
        da_luc_lastyear_dic, da_luc_1year_dic, 
        da_road_distance_dic, da_boundary_dis_dic, 
        da_npp_1year_dic, da_npp_lastyear_dic
    )

def trans_to_no_coords(da_luc_lastyear_dic, radius, ar_circle):
    da_combine_no_coords_lastyear = xr.DataArray(
        [i.values for i in da_luc_lastyear_dic.values()], 
        coords={"idx": [i for i in da_luc_lastyear_dic.keys()], "y": np.arange(-radius, radius+1)[::-1], "x": np.arange(-radius, radius+1)}
    )
    da_combine_no_coords_lastyear = xr.where(ar_circle==1, da_combine_no_coords_lastyear, np.nan)
    return  da_combine_no_coords_lastyear

def point_to_sample_da(gdf_conflict_1year, da_luc, da_road_dis, da_boundary_dis, da_npp_1year, da_npp_lastyear, radius, ar_circle):
    da_luc_lastyear_dic, da_luc_1year_dic, da_road_distance_dic, da_boundary_dis_dic, da_npp_1year_dic, da_npp_lastyear_dic = extract_circle(gdf_conflict_1year, da_luc, da_road_dis, da_boundary_dis, da_npp_1year, da_npp_lastyear, radius)

    da_combine_no_coords_lastyear = trans_to_no_coords(da_luc_lastyear_dic, radius, ar_circle)
    da_combine_no_coords_1year = trans_to_no_coords(da_luc_1year_dic, radius, ar_circle)
    da_combine_no_coords_road_dis = trans_to_no_coords(da_road_distance_dic, radius, ar_circle)
    da_combine_no_coords_road_dis = xr.where(da_combine_no_coords_road_dis < 0, np.nan, da_combine_no_coords_road_dis)

    da_combine_no_coords_boundary_dis = trans_to_no_coords(da_boundary_dis_dic, radius, ar_circle)
    da_combine_no_coords_boundary_dis = xr.where(da_combine_no_coords_boundary_dis < 0, np.nan, da_combine_no_coords_boundary_dis)
    
    da_combine_no_coords_npp_lastyear = trans_to_no_coords(da_npp_lastyear_dic, radius, ar_circle)
    da_combine_no_coords_npp_1year = trans_to_no_coords(da_npp_1year_dic, radius, ar_circle)

    return (
        da_combine_no_coords_lastyear, da_combine_no_coords_1year, 
        da_combine_no_coords_road_dis, da_combine_no_coords_boundary_dis,
        da_combine_no_coords_npp_lastyear, da_combine_no_coords_npp_1year
    )

In [ ]:
# LandScan
def extract_circle_population(year, gdf_conflict_1year, radius, ar_circle):
    da_pop = xr.open_dataarray(path_data / f"pop/{year}.tif").sel(band=1).drop("band")
    da_pop_dic = {}
    for idx, x, y in tqdm(gdf_conflict_1year[["idx", "x", "y"]].values):

        y_, x_ = get_circle_locations(y, x, da_pop, radius)
        da_pop_dic[idx] = da_pop.isel(x=x_, y=y_)

    da_combine_no_coords_pop = trans_to_no_coords(da_pop_dic, radius, ar_circle)
    return da_combine_no_coords_pop

In [ ]:
def pipline_sample(year, radius, ar_circle):
    gdf_conflict_1year = get_gdf_conflict_1year(year)
    gdf_miss_conflict = get_miss_conflict_random_point(year, gdf_conflict_1year)

    da_luc = load_luc(year)
    da_npp_1year, da_npp_lastyear = load_npp(year)
        
    da_conflict_lst = point_to_sample_da(gdf_conflict_1year, da_luc, da_road_dis, da_boundary_dis, da_npp_1year, da_npp_lastyear, radius, ar_circle)
    for da_, name_ in zip(da_conflict_lst, ["luc_lastyear", "luc_currentyear", "road_dis", "boundary_dis", "npp_lastyear", "npp_currentyear"]):
        da_.to_netcdf(path_data / f"PSM{radius}km/conflict_sample/{year}_{name_}.nc")

    da_non_conflict_lst = point_to_sample_da(gdf_miss_conflict, da_luc, da_road_dis, da_boundary_dis, da_npp_1year, da_npp_lastyear, radius, ar_circle)
    for da_, name_ in zip(da_non_conflict_lst, ["luc_lastyear", "luc_currentyear", "road_dis", "boundary_dis", "npp_lastyear", "npp_currentyear"]):
        da_.to_netcdf(path_data / f"PSM{radius}km/non_conflict_sample/{year}_{name_}.nc")
        
    return  (gdf_conflict_1year, gdf_miss_conflict), (da_conflict_lst, da_non_conflict_lst)


In [ ]:
da_circle, ar_circle = get_ar_circle(radius)
(path_data / f"PSM{radius}km/non_conflict_sample").mkdir(parents=True, exist_ok=True)
(path_data / f"PSM{radius}km/conflict_sample").mkdir(parents=True, exist_ok=True)

In [ ]:
for year in range(2002, 2024):
    if not (path_data / f"PSM{radius}km/non_conflict_sample/{year}_npp_currentyear.nc").exists():
        (gdf_conflict_1year, gdf_miss_conflict), (da_conflict_lst, da_non_conflict_lst) = pipline_sample(year, radius, ar_circle)
    
    if not (path_data / f"PSM{radius}km/non_conflict_sample/{year}_pop.nc").exists():
        gdf_conflict_1year = get_gdf_conflict_1year(year)
        gdf_miss_conflict = get_miss_conflict_random_point(year, gdf_conflict_1year)

        da_combine_no_coords_pop_con = extract_circle_population(year, gdf_conflict_1year, radius, ar_circle)
        da_combine_no_coords_pop_non_con = extract_circle_population(year, gdf_miss_conflict, radius, ar_circle)

        da_combine_no_coords_pop_con.to_netcdf(path_data / f"PSM{radius}km/conflict_sample/{year}_pop.nc")
        da_combine_no_coords_pop_non_con.to_netcdf(path_data / f"PSM{radius}km/non_conflict_sample/{year}_pop.nc")